# Vector 2: Geometries, Spatial Operations and Visualization Demo

UW Geospatial Data Analysis  
CEE498/CEWA599  
David Shean  

## Background

https://autogis-site.readthedocs.io/en/latest/notebooks/L1/geometric-objects.html

Take a look at the first few sections of the shapely manual: https://shapely.readthedocs.io/en/stable/manual.html

Many of these are implemented in GeoPandas, and they will operate over GeoDataFrame and GeoSeries objects:
https://geopandas.org/en/stable/docs/reference.html

These are common functions, and good to have in your toolkit for additional spatial analysis
If you've taken a GIS class, you've definitely encountered these, probably through some kind of vector toolkit

## Interactive Discussion
### GeoDataFrame vs. GeoSeries
* https://geopandas.org/data_structures.html
* Indexing and selection - `iloc`, `loc`
* Pandas `squeeze`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.squeeze.html

### Geometry objects
* POINT, LINE, POLYGON
* Polygon vs. MultiPolygon

### GEOS geometry operations, as exposed by `shapely`
* GEOS https://trac.osgeo.org/geos/
* https://geopandas.org/geometric_manipulations.html
* Intersection
* Union
* Buffer

### Spatial joins with GeoPandas
* https://gisgeography.com/spatial-join/
* https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

### Visualization: Chloropleth and Heatmap
* https://geopandas.org/mapping.html

### Interactive plotting
* ipyleaflet, folium
* Basemap tiles with contextily

## Interactive Demo

In [ ]:
### Polygon creation
### Basic geometric operations on GeoDataFrame and Geometry objects

In [ ]:
import os
import requests

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import fiona
#plt.rcParams['figure.figsize'] = [10, 8]

In [ ]:
#%matplotlib widget

## Read in the projected GLAS points
* Ideally, use the file with equal-area projection exported during Lab04
* Note, you can right-click on a file in the Jupyterlab file browser, and select "Copy Path", then paste, but make sure you get the correct relative path to the current notebook (`../`)
* If you have issues with your file, you can recreate:
    * Read the original GLAS csv
    * Load into GeoDataFrame, define CRS (`'EPSG:4326'`)
    * Reproject with following PROJ string: `'+proj=aea +lat_1=37.00 +lat_2=47.00 +lat_0=42.00 +lon_0=-114.27'`

In [ ]:
#Loading from Lab04
#aea_fn = '../04_Vector1_Geopandas_CRS_Proj/conus_glas_aea.gpkg'
#glas_gdf_aea = gpd.read_file(aea_fn)

In [ ]:
#Recreating
aea_proj_str = '+proj=aea +lat_1=37.00 +lat_2=47.00 +lat_0=42.00 +lon_0=-114.27'
csv_fn = '../01_Shell_Github/data/GLAH14_tllz_conus_lulcfilt_demfilt.csv'
glas_df = pd.read_csv(csv_fn)
glas_gdf = gpd.GeoDataFrame(glas_df, crs='EPSG:4326', geometry=gpd.points_from_xy(glas_df['lon'], glas_df['lat']))
glas_gdf_aea = glas_gdf.to_crs(aea_proj_str)

In [ ]:
glas_gdf_aea.head()

## Create a variable to store the `crs` of your GeoDataFrame
* Quickly print this out to verify everything looks good

In [ ]:
aea_crs = glas_gdf_aea.crs
aea_crs

## Read in the state polygons

In [ ]:
states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_5m.json'
#states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_500k.json'
states_gdf = gpd.read_file(states_url)

## Limit to Lower48

In [ ]:
idx = states_gdf['NAME'].isin(['Alaska','Puerto Rico','Hawaii'])
states_gdf = states_gdf[~idx]

## Reproject the states to match the `crs` of your points

In [ ]:
states_gdf_aea = states_gdf.to_crs(aea_crs)

## Create a quick plot to verify everything looks good
* Can re-use plotting code near the end of Lab04

In [ ]:
f, ax = plt.subplots()
states_gdf_aea.plot(ax=ax, facecolor='none', edgecolor='k')
glas_gdf_aea.plot(ax=ax, column='glas_z', cmap='inferno', markersize=1, legend=True);

## Extract the MultiPolygon geometry object for Washington from your reprojected states GeoDataFrame
* Use the state 'NAME' value to isolate the approprate GeoDataFrame record for Washington
* Assign the `geometry` attribute for this record to a new variable called `wa_geom`
    * This is a little tricky
    * After a boolean filter to get the WA record, you will need to use something like `iloc[0]` to extract a GeoSeries, and then isolate the `geometry` attribute
        * `wa_geom = wa_gdf.iloc[0].geometry`
    * Use the python `type()` function to verify that your output type is `shapely.geometry.multipolygon.MultiPolygon`

In [ ]:
# This is a new GeoDataFrame with one entry
wa_gdf = states_gdf_aea[states_gdf_aea['NAME'] == 'Washington']

In [ ]:
wa_gdf

In [ ]:
type(wa_gdf)

### Review loc, iloc, squeeze

In [ ]:
wa_gdf.loc[47]

In [ ]:
wa_gdf.iloc[0]

In [ ]:
wa_gdf.squeeze()

In [ ]:
type(wa_gdf.squeeze())

In [ ]:
# This is the Geometry object for that one entry
wa_geom = wa_gdf.squeeze().geometry

In [ ]:
type(wa_geom)

## Inspect the Washington geometry object

### What happens when you pass the geometry object to `print()`?

In [ ]:
#print(wa_geom)

### What happens when you execute a notebook cell containing only the geometry object variable name? Oooh.
* Thanks Jupyter/iPython!

In [ ]:
wa_geom

### What is the geometry type?

In [ ]:
wa_geom.geom_type

## Find the geometric center of WA state
* See the `centroid` attribute
* You may have to `print()` this to see the coordinates

In [ ]:
#GeoDataFrame
#c = wa_gdf.centroid.iloc[0]
#MultiPolygon Geometry
c = wa_geom.centroid

In [ ]:
c

In [ ]:
print(c)

### How many individual polygons are contained in the WA geometry?  
* Hint: how would you get the number of elements in a standard Python tuple or list?  
* If more than one, why?

In [ ]:
list(wa_geom.geoms)

In [ ]:
len(wa_geom.geoms)

## Cracking open the geometry collection

### Compute the area of each individual polygon
* Remember, this MultiPolygon object is iterable, so maybe list comprehension here?
* Store the output areas in a new list or array

In [ ]:
poly_area = [x.area for x in wa_geom.geoms]
poly_area

### Isolate and render the polygons that have min and max area
* Remember the NumPy `argmax` function? Maybe useful here...

In [ ]:
maxidx = np.argmax(poly_area)
minidx = np.argmin(poly_area)

In [ ]:
maxidx

In [ ]:
print(wa_geom.geoms[maxidx].area)
wa_geom.geoms[maxidx]

In [ ]:
print(wa_geom.geoms[minidx].area)
wa_geom.geoms[minidx]

### How many vertices are in the largest polygon?
* Let's start by looking at the `exterior` ring of the largest polygon geometry
    * This is actually a line, so if you ever need to convert a simple polygon geometry to a line geometry, now you know how to do it - use `exterior`! 
    * You should see an outline of WA state
* Now let's access the coordinates for this line geometry with `coords[:]`
    * This will return a list of (x,y) tuples for each vertex
* You already know how to determine the number of items in a list!

In [ ]:
wa_geom.geoms[maxidx].exterior

In [ ]:
type(wa_geom.geoms[maxidx].exterior)

In [ ]:
wa_geom.geoms[maxidx].exterior.coords

In [ ]:
#Preview first 10 vertices
wa_geom.geoms[maxidx].exterior.coords[:][0:10]

In [ ]:
# Note -1 to remove repeated coord needed to close the polygon
len(wa_geom.geoms[maxidx].exterior.coords[:]) - 1

## How many vertices in the smallest polygon?

In [ ]:
len(wa_geom.geoms[minidx].exterior.coords[:]) - 1

### Take a look at the list of (x,y) coordinates in the smallest polygon
* What do you notice about the first and last coordinate?

In [ ]:
wa_geom.geoms[minidx].exterior.coords[:]

This is a closed polygon!  It starts and ends at the same point.  So technically, you have one less vertex than the total number of points in the polygon.

## Determine the perimeter of WA state in km
* This should be quick - just use an attribtue of the MultiPolygon geometry

In [ ]:
wa_geom.length/1E3

## Explore the `simplify()` method for your MultiPolygon
* This can be very useful if you have complex geometry objects that require a lot of memory
    * Perhaps a line from a GPS track, or a polygon extracted from a raster with a vertex at each pixel
    * You can simplify, preserve almost all of the original information, and remove many (sometimes most) of the redundant/unnecessary vertices
* https://shapely.readthedocs.io/en/latest/manual.html#object.simplify
* Need to provide a `tolerance` in units of meters
    * Try 100, 1000, 10000, 100000
    * How does this affect the perimeter measurement?

In [ ]:
def smart_simplify(geom, tol):
    newgeom = geom.simplify(tol)
    newvertcount = sum(len(x.exterior.coords[:]) for x in newgeom)
    gpd.GeoSeries(newgeom).plot(figsize=(4,3))
    print(tol, '%0.1f' % (newgeom.length/1E3), newvertcount)

In [ ]:
print("Tolerance", "Perimieter", "Num vertices")
for i in (100,1000,10000,100000):
    smart_simplify(wa_geom, i)

### Extra Credit: What percentage of the total WA state perimeter is from islands?

In [ ]:
(1.0 - (wa_geom[maxidx].length / wa_geom.length))*100

## Unite the West Coast!
* Let's create a single Multipolygon for West Coast states: Washington, Oregon and California
* Start by extracting those states to a new GeoDataFrame - can use the `isin()` function for Pandas, which is similar to built-in `in` operation in Python

In [ ]:
idx = states_gdf_aea['NAME'].isin(['Washington','Oregon','California'])
westcoast_gdf = states_gdf_aea[idx]
westcoast_gdf

In [ ]:
westcoast_gdf.plot();

## Combine the states as a single MultiPolygon geometry
* See the `unary_union` attribute 

In [ ]:
westcoast_geom = westcoast_gdf.unary_union
westcoast_geom

Note: If you have a column that classifies features in a GeoDataFrame (e.g., a column for `region` with a shared value of 'West Coast' for these polygons), you can also use `dissolve(by='region')` to create a new GeoDataFrame of merged polygons

## Now buffer the combined geometry
* Use a 50 km buffer

In [ ]:
westcoast_geom_buff = westcoast_geom.buffer(50000)
westcoast_geom_buff

In [ ]:
#Buffer distance can be negative
westcoast_geom_erode = westcoast_geom.buffer(-50000)
westcoast_geom_erode

## Use a `difference` operation to isolate the buffered region around the individual polygons
* This is sometimes useful if you need to extract statistics from another dataset

In [ ]:
westcoast_geom_buff.difference(westcoast_geom_erode)

In [ ]:
westcoast_geom_buff.difference(westcoast_geom)

### Somebody should put that on a t-shirt!

## RGI glacier polygons

Let's grab some glacier outline poygons from the Randolph Glacier Inventory (RGI) v6.0: https://www.glims.org/RGI/

In [ ]:
#Fetch the zip file for Region 02 (Western North America)
rgi_zip_fn = '02_rgi60_WesternCanadaUS.zip'

if not os.path.exists(rgi_zip_fn):
    url = 'https://www.glims.org/RGI/rgi60_files/' + rgi_zip_fn
    myfile = requests.get(url)
    open(rgi_zip_fn, 'wb').write(myfile.content)

In [ ]:
#Specify the shapefile filename within zip archive
rgi_fn = 'zip://02_rgi60_WesternCanadaUS.zip!02_rgi60_WesternCanadaUS.shp'

## Load RGI shapefile using Geopandas
* Very easy with `read_file()` meethod

In [ ]:
rgi_gdf = gpd.read_file(rgi_fn)

In [ ]:
rgi_gdf.head()

That's it!

In [ ]:
#By default a new integer index is created.  Can use the RGI ID as our index
#rgi_gdf = rgi_gdf.set_index('RGIId')

## Create a quick plot

In [ ]:
#rgi_gdf.explore()

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world.plot(alpha=0.2)
states_gdf.plot(ax=ax, facecolor='none', edgecolor='0.5', linewidth=0.5)
rgi_gdf.plot(ax=ax, edgecolor='k', linewidth=0.5);

## Clip RGI polygons to WA state
GeoPandas makes spatial selection easy.  

We'll have two options: 1) using a bounding box, and 2) using an arbitrary polygon.

#### 1. Bounding box

In [ ]:
xmin, ymin, xmax, ymax = wa_gdf.to_crs('EPSG:4326').total_bounds

#### Create new GeoDataFrame from output of simple spatial filter with GeoPandas `cx` function
* https://geopandas.org/indexing.html

In [ ]:
rgi_gdf_wa = rgi_gdf.cx[xmin:xmax, ymin:ymax]

In [ ]:
print("Number of RGI polygons before:",rgi_gdf.shape[0])
rgi_gdf_wa = rgi_gdf[rgi_gdf_idx]
print("Number of RGI polygons after:", rgi_gdf_wa.shape[0])

#### Quick plot to verify

In [ ]:
rgi_gdf_wa.plot(edgecolor='k', linewidth=0.5);

#### 2. Clip points to arbitrary Polygon geometry

In [ ]:
wa_gdf_chull = wa_gdf.to_crs('EPSG:4326').unary_union.convex_hull

In [ ]:
#Check the type
type(wa_gdf_chull)

#### Preview geometry
Note that geometry objects (points, lines, polygons, etc.) will render directly in the Jupyter notebook!  Great for quick previews.

In [ ]:
wa_gdf_chull

In [ ]:
print(wa_gdf_chull)

#### Compute intersection between all RGI polygons and the convex hull
Use the GeoDataFrame `intersects()` function.  
This will return a Boolean DataSeries, True if points intersect the polygon, False if they do not

In [ ]:
rgi_gdf_idx = rgi_gdf.intersects(wa_gdf_chull)

In [ ]:
rgi_gdf_idx

#### Extract records with True for the intersection

In [ ]:
print("Number of RGI polygons before:",rgi_gdf.shape[0])
rgi_gdf_wa = rgi_gdf[rgi_gdf_idx]
print("Number of RGI polygons after:", rgi_gdf_wa.shape[0])

#### Quick plot to verify
Note latitude range

In [ ]:
rgi_gdf_wa.plot(edgecolor='k', linewidth=0.5);

In [ ]:
rgi_gdf_wa.explore()

In [ ]:
rgi_gdf_wa_aea = rgi_gdf_wa.to_crs(aea_crs)

In [ ]:
rgi_gdf_wa_aea.geometry.centroid.x

In [ ]:
f, ax = plt.subplots(figsize=(6,6))
x = rgi_gdf_wa_aea.geometry.centroid.x
y = rgi_gdf_wa_aea.geometry.centroid.y
hb = ax.hexbin(x, y, gridsize=20, mincnt=1)
ax.set_aspect('equal')

## Merge GLAS points and RGI polygons

Earlier we computed some statistics for the full CONUS GLAS sample and hex bins.  Now let's analyze the GLAS points that intersect each RGI glacier polygon. 

One approach would be to loop through each glacier polygon, and do an intersection operation with all points.  But this is inefficient, and doesn't scale well.  It is much more efficient to do a spatial join between the points and the polygons, then groupby and aggregate to compute the relevant statistics for all points that intersect each glacier polygon.

You may have learned how to perform a join or spatial join in a GIS course.  So, do we need to open ArcMap or QGIS here?  Do we need a full-fledged spatial database like PostGIS?  No!  GeoPandas has you covered.

* Start by reviewing the Spatial Join documentation here: http://geopandas.org/mergingdata.html
* Use the geopandas `sjoin` method: http://geopandas.org/reference/geopandas.sjoin.html

## First, we need to make sure all inputs have the same projection
* Reproject the RGI polygons to match our point CRS (custom Albers Equal-area)

In [ ]:
glas_gdf_aea.crs

In [ ]:
rgi_gdf_wa_aea.crs

### Optional: isolate relevant columns to simplify our output

In [ ]:
glas_gdf_aea.columns

In [ ]:
rgi_gdf_wa_aea.columns

In [ ]:
glas_col = ['decyear', 'glas_z', 'geometry']
rgi_col = ['RGIId', 'Area', 'Name', 'geometry']

glas_gdf_aea_lite = glas_gdf_aea[glas_col]
rgi_gdf_wa_aea_lite = rgi_gdf_wa_aea[rgi_col]

In [ ]:
glas_gdf_aea_lite

In [ ]:
rgi_gdf_wa_aea_lite

## Now try a spatial join between these two 
* Use the GLAS points as the "left" GeoDataFrame and the RGI polygons as the "right" GeoDataFrame
* Start by using default options (`op='intersects', how='inner'`)
* Note the output geometry type and columns

In [ ]:
glas_gdf_aea_rgi = gpd.sjoin(glas_gdf_aea_lite, rgi_gdf_wa_aea_lite)
glas_gdf_aea_rgi

## Check number of records

In [ ]:
print("Number of RGI polygons before:", rgi_gdf_wa_aea_lite.shape[0])
print("Number of GLAS points before:", glas_gdf_aea.shape[0])
print("Number of GLAS points that intersect RGI polygons:", glas_gdf_aea_rgi.shape[0])

## Check number of GLAS points per RGI polygon

In [ ]:
glas_gdf_aea_rgi['RGIId'].value_counts()

## Which glacier has the greatest number of points?

Some notes on indexing and selecting from Pandas DataFrame: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/

Here we'll use the `iloc` function to pull out the record for the RGIId key with the highest point count.

In [ ]:
label = glas_gdf_aea_rgi['RGIId'].value_counts().index[0]
print(label)

In [ ]:
rgi_maxcount = rgi_gdf_wa_aea_lite[rgi_gdf_wa_aea_lite['RGIId'] == label].iloc[0]
rgi_maxcount

In [ ]:
rgi_maxcount.geometry

In [ ]:
glas_gdf_aea_rgi.explore()